In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from joblib import Parallel, delayed

# Grid parameters
grid_h, grid_w = 50, 50
radius = 1
n_points = 8 * radius

def extract_features_from_cell(cell):
    features = []
    features.append(np.mean(cell))
    features.append(np.std(cell))
    
    lbp = local_binary_pattern(cell, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points+3), density=True)
    features.extend(hist)
    
    edges = cv2.Canny(cell, 50, 150)
    features.append(np.sum(edges > 0) / cell.size)
    
    return np.array(features)

def extract_features_from_image(img):
    features_list = []
    h, w = img.shape
    for y in range(0, h, grid_h):
        for x in range(0, w, grid_w):
            cell = img[y:y+grid_h, x:x+grid_w]
            if cell.shape[0] != grid_h or cell.shape[1] != grid_w:
                cell = cv2.resize(cell, (grid_w, grid_h))
            features_list.append(extract_features_from_cell(cell))
    return np.vstack(features_list)

def load_and_process_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, 0, None
    img = cv2.resize(img, (800, 600))
    feats = extract_features_from_image(img)
    return feats, feats.shape[0], img

def load_images_parallel(image_folder, n_jobs=-1):
    paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    results = Parallel(n_jobs=n_jobs)(delayed(load_and_process_image)(p) for p in paths)
    
    all_features = []
    grid_indices = []
    images = []
    for idx, (feats, n_grids, img) in enumerate(results):
        if feats is not None:
            all_features.append(feats)
            grid_indices.append((idx, n_grids))
            images.append(img)
    if all_features:
        all_features = np.vstack(all_features)
    else:
        all_features = np.array([])
    return all_features, grid_indices, images, paths

def draw_grid_overlay(img, grid_map):
    overlay = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    num_rows, num_cols = grid_map.shape
    for i in range(num_rows):
        for j in range(num_cols):
            y1, x1 = i*grid_h, j*grid_w
            y2, x2 = y1+grid_h, x1+grid_w
            color = (0,255,0) if grid_map[i,j]==1 else (0,255,255)  # green for detected wildlife, yellow for others
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, 1)
    return overlay

# Main pipeline
if __name__ == "__main__":
    image_folder = "images"
    output_folder = "output"
    os.makedirs(output_folder, exist_ok=True)

    all_features, grid_indices, images, paths = load_images_parallel(image_folder, n_jobs=-1)

    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(all_features)

    iso = IsolationForest(contamination=0.1, random_state=42, n_jobs=-1)
    iso.fit(all_features_scaled)
    preds = iso.predict(all_features_scaled)

    start = 0
    for img_idx, n_grids in grid_indices:
        grid_pred = preds[start:start+n_grids]
        start += n_grids
        num_rows = 600 // grid_h
        num_cols = 800 // grid_w
        grid_map = (grid_pred == -1).astype(int).reshape((num_rows, num_cols))

        overlay_img = draw_grid_overlay(images[img_idx], grid_map)
        filename = os.path.basename(paths[img_idx])
        cv2.imwrite(os.path.join(output_folder, f"overlay_{filename}"), overlay_img)

        # Optional: display the first 5 images
        if img_idx < 5:
            cv2.imshow(f"Overlay {img_idx}", overlay_img)
            cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(f"Overlay images saved in folder: {output_folder}")
